# Inserting ISS Data To MySQL with SQLAlchemy

## Collecting data from the API

In [ ]:
!pip install pymsql

ERROR: Could not find a version that satisfies the requirement pymsql (from versions: none)
ERROR: No matching distribution found for pymsql


In [ ]:
import requests
url =  "http://api.open-notify.org/iss-now.json"
response = requests.get(url)
json_response = response.json()
json_response

{'iss_position': {'longitude': '1.3260', 'latitude': '26.7644'},
 'timestamp': 1654766328,
 'message': 'success'}

Extract only the pieces of info we need and store them in a dictionary

In [ ]:
data_to_insert = {"latitude":json_response["iss_position"]["latitude"],
                  "longitude":json_response["iss_position"]["longitude"],
                  "iss_timestamp":json_response["timestamp"]}
data_to_insert

{'latitude': '26.7644', 'longitude': '1.3260', 'iss_timestamp': 1654766328}

Timestamp from UNIX to proper datetime format

In [ ]:
from datetime import datetime
data_to_insert["iss_timestamp"] = datetime.utcfromtimestamp(data_to_insert["iss_timestamp"]).strftime('%Y-%m-%d %H:%M:%S')
data_to_insert

{'latitude': '26.7644',
 'longitude': '1.3260',
 'iss_timestamp': '2022-06-09 09:18:48'}

Create a pandas Dataframe with the same columns as the SQL table we want to have in our database

In [ ]:
import pandas as pd

iss_df = pd.DataFrame({
    "latitude":[],
    "longitude":[],
    "iss_timestamp":[]
})

Append data to DataFrame

In [ ]:
iss_df = iss_df.append(data_to_insert, ignore_index=True)
iss_df

,latitude,longitude,iss_timestamp
0,26.7644,1.3260,2022-06-09 09:18:48


# SQLAlchemy

In [ ]:
import sqlalchemy # install if needed

Specify MySQL connection. You need to previously create the schema (also called database in MySQL) `iss_workshop` on your local instance of MySQL. 

You can do that with MySQLWorkbench by connecting to your local instance and typing `CREATE DATABASE iss_workshop;` in a new query tab.

Use pandas method `to_sql` with the argument `if_exists=append` to create the table (only the first time we run it) and insert the new rows into it.

In [ ]:
iss_df.to_sql('iss_position', 
              if_exists='append', 
              con=con, 
              index=False)

ModuleNotFoundError: No module named 'pymysql'

Check on MySQLWorkbench that a new table `iss_position` exists within the `iss_workshop` database, and that a new row has been inserted on it. If you run the whole notebook again, another row should appear there.